In [ ]:
import pandas as pd
import numpy as np
from torch import nn


In [ ]:
#read in only the first 10000 rows:
df = pd.read_parquet('df_post.parquet')
#adding POL_TheftClaimRecency

#Done in the preprocessor.
# df['POL_TheftClaimRecency_ORD'] = pd.to_numeric(df['POL_TheftClaimRecency']).copy(deep=True)

# df['POL_TheftClaimRecency_CAT'] = 'LESS'
# catlist = [8, '8', '8.0']

# df['POL_TheftClaimRecency_CAT'] = df['POL_TheftClaimRecency_CAT'].mask(df['POL_TheftClaimRecency_ORD'].isin(catlist), 'MORE')


In [ ]:
df['Frequency'].describe()

In [ ]:
import preprocessor as pp  

%reload_ext autoreload
%autoreload 2

In [ ]:
pre = pp.Preprocessor(df, target_col = 'Acc_GrossClaim')


In [ ]:
df['Frequency'].describe()

In [ ]:
0.09

In [ ]:
pre.df['Acc_GrossClaimClipped'].value_counts()

In [ ]:
pre.df['Acc_GrossClaim'].value_counts()

In [ ]:
import wandb

In [ ]:
api = wandb.Api()   

In [ ]:
# artifact = run.use_artifact('middelman/catboost_rossmann_mse/catboost_regressor:v34', type='model')
# artifact_dir = artifact.download()

saint_project_name = "middelman/saint_rossmann_mse"
saint_metric_name = "orig_valid_rmse"

saint_runs = api.runs(saint_project_name)

saint_sorted_runs = sorted(saint_runs, key=lambda r: r.summary.get(saint_metric_name, float('inf')))


In [ ]:
saint_best_run = saint_sorted_runs[0]
import pprint
pprint.pprint(saint_best_run.summary)

In [ ]:
opt = saint_best_run.config

In [ ]:
#update opt patience to 5
opt['patience'] = 5

In [ ]:
pprint.pprint(opt)

In [ ]:
from models import SAINT


In [ ]:
cat_idxs = pre.cat_idxs
cat_dims = pre.cat_dims
cont_idxs = pre.ord_idxs

In [ ]:
cat_dims_saint = np.append(np.array([1]), np.array(cat_dims)).astype(int) #Appending 1 for CLS token, this is later used to generate embeddings.


In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using {}".format(device))

model = SAINT(categories = tuple(cat_dims_saint), 
            num_continuous = len(cont_idxs),                
            dim = opt['embedding_size'],                           
            dim_out = 1,                       
            depth = opt['transformer_depth'],                       
            heads = opt['attention_heads'],                         
            attn_dropout = opt['attention_dropout'],             
            ff_dropout = opt['ff_dropout'],                  
            mlp_hidden_mults = (4, 2),       
            cont_embeddings = opt['cont_embeddings'],
            attentiontype = opt['attentiontype'],
            final_mlp_style = opt['final_mlp_style'],
            y_dim = 1
            )

In [ ]:
import gc
gc.collect()

In [ ]:
import torch.optim as optim

# criterion = nn.MSELoss().to(device)
criterion = nn.PoissonNLLLoss().to(device)
model.to(device)
optimizer = optim.AdamW(model.parameters()
                        # , lr=opt['lr']
                        , lr = 0.00005
                        )

In [ ]:
from augmentations import embed_data_mask
from utils import count_parameters, classification_scores, mean_sq_error, get_loss
import os
vision_dset = opt['vision_dset']


In [ ]:
import pprint
pprint.pprint(opt)

In [ ]:
run = True
i = 1
run = wandb.init(project="mjolnir_saint_rmse", entity="middelman", group='poisson', name="saint_rev1_"+str(i/5*100) + '%_of_data')


In [ ]:
pre.target_col

In [ ]:
pre.df[pre.target_col].describe()

In [ ]:
pre.df[pre.target_col].value_counts()

In [ ]:
pre.df[pre.target_col].isna().sum()

In [ ]:
pre.df[pre.target_col].fillna(0, inplace=True)

In [ ]:
pre.target_col

In [ ]:
if run:
    early_stop_count = 0
    for n in np.arange(1, 2, 1):
        print(f'Loading only the first {i/5*100}% of training data')
        # run = wandb.init(project="mjolnir_saint_rmse", entity="middelman", group='poisson', name="saint_rev0_"+str(i/5*100) + '%_of_data')
        trainloader, validloader, testloader = pre.preprocess_for_saint(batch_size = opt['batchsize'], load_percentage = i/5)
        best_valid_auroc = 0
        best_valid_accuracy = 0
        best_test_auroc = 0
        best_test_accuracy = 0
        best_valid_rmse = 100000
        best_valid_nllloss = 100000
        print('Using the optimizer: ', opt['optimizer'])
        print('Training begins now.')
        for epoch in range(opt['epochs']):
            print('Starting epoch: ', epoch)
            model.train()
            running_loss = 0.0
            for _, data in enumerate(trainloader, 0):
                optimizer.zero_grad()
                # x_categ is the the categorical data, x_cont has continuous data, y_gts has ground truth ys. cat_mask is an array of ones same shape as x_categ and an additional column(corresponding to CLS token) set to 0s. con_mask is an array of ones same shape as x_cont. 
                x_categ, x_cont, y_gts, cat_mask, con_mask = data[0].to(device), data[1].to(device), data[2].to(device), data[3].to(device), data[4].to(device)

                # We are converting the data to embeddings in the next step
                _ , x_categ_enc, x_cont_enc = embed_data_mask(x_categ, x_cont, cat_mask, con_mask, model)           
                reps = model.transformer(x_categ_enc, x_cont_enc)
                # select only the representations corresponding to CLS token and apply mlp on it in the next step to get the predictions.
                y_reps = reps[:,0,:]
                
                y_outs = model.mlpfory(y_reps)
                if opt['task'] == 'regression':
                    loss = criterion(y_outs, y_gts) 
                else:
                    loss = criterion(y_outs,y_gts.squeeze()) 
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                
                # print(running_loss)
                if opt['active_log']:
                    wandb.log({'epoch': epoch ,'running_loss': running_loss, 
                    'train_loss': loss.item()
                    })
                print('Loss: ', loss.item())
                # if epoch%1==0:
                model.eval()
                with torch.no_grad():
                    
                    #Get validation loss
                    valid_loss = get_loss(model, validloader, device, criterion)
                    test_loss = get_loss(model, testloader, device, criterion)
                    if opt['active_log']:
                        wandb.log(
                            {'epoch': epoch,
                        'valid_loss': valid_loss.item(),
                        'test_loss': test_loss.item()
                            }
                        )
                    
                    print('Valid loss: ', valid_loss.item())
                    print('Test loss: ', test_loss.item())
                    
                    if valid_loss.item() < best_valid_nllloss:
                        best_valid_nllloss = valid_loss.item()
                        
                        #get the run id from wandb
                        run_id = wandb.run.id
                        torch.save(model.state_dict(), f'models/mjolnir4/SAINT_model_best_{run_id}.pt')
                        #Save as artifact on weights and biases
                        artifact = wandb.Artifact(f'SAINT_model_best_{run_id}', type='model')
                        #add the run id to the artifact                       
                        # wandb.run.log_artifact(artifact)
                        early_stop_count = 0
                    else:
                        early_stop_count += 1
                        print(f"Early stopping counter: {early_stop_count}/{opt['patience']}")
                        if early_stop_count >= opt['patience']:
                            print('EARLY STOPPING')
                            break
                    model.train()
                



In [ ]:
opt['patience']

In [ ]:
run = True

if run:
    for i in np.arange(4, 6, 1):
        print(f'Loading only the first {i/5*100}% of training data')
        run = wandb.init(project="mjolnir_saint_rmse", entity="middelman", name="saint_rev0_"+str(i/5*100) + '%_of_data')
        trainloader, validloader, testloader = pre.preprocess_for_saint(batch_size = opt['batchsize'], load_percentage = i/5)
        best_valid_auroc = 0
        best_valid_accuracy = 0
        best_test_auroc = 0
        best_test_accuracy = 0
        best_valid_rmse = 100000
        print('Using the optimizer: ', opt['optimizer'])
        print('Training begins now.')
        early_stop_count = 0
        for epoch in range(opt['epochs']):
            print('Starting epoch: ', epoch)
            model.train()
            running_loss = 0.0
            for i, data in enumerate(trainloader, 0):
                optimizer.zero_grad()
                # x_categ is the the categorical data, x_cont has continuous data, y_gts has ground truth ys. cat_mask is an array of ones same shape as x_categ and an additional column(corresponding to CLS token) set to 0s. con_mask is an array of ones same shape as x_cont. 
                x_categ, x_cont, y_gts, cat_mask, con_mask = data[0].to(device), data[1].to(device), data[2].to(device), data[3].to(device), data[4].to(device)

                # We are converting the data to embeddings in the next step
                _ , x_categ_enc, x_cont_enc = embed_data_mask(x_categ, x_cont, cat_mask, con_mask, model)           
                reps = model.transformer(x_categ_enc, x_cont_enc)
                # select only the representations corresponding to CLS token and apply mlp on it in the next step to get the predictions.
                y_reps = reps[:,0,:]
                
                y_outs = model.mlpfory(y_reps)
                if opt['task'] == 'regression':
                    loss = criterion(y_outs, y_gts) 
                else:
                    loss = criterion(y_outs,y_gts.squeeze()) 
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            # print(running_loss)
            if opt['active_log']:
                wandb.log({'epoch': epoch ,'train_epoch_loss': running_loss, 
                'loss': loss.item()
                })
            if epoch%1==0:
                    model.eval()
                    with torch.no_grad():
                        valid_rmse = mean_sq_error(model, validloader, device, vision_dset)    
                        test_rmse = mean_sq_error(model, testloader, device, vision_dset)  
                        train_rmse = mean_sq_error(model, trainloader, device, vision_dset)  
                        print('[EPOCH %d] VALID RMSE: %.3f' %
                            (epoch + 1, valid_rmse))# , orig_valid_rmse ))
                        print('[EPOCH %d] TEST RMSE: %.3f' %
                            (epoch + 1, test_rmse))#, orig_test_rmse ))
                        print('[EPOCH %d] TRAIN RMSE: %.3f' %
                            (epoch + 1, train_rmse)) #, orig_train_rmse ))
                        
                        if opt['active_log']:
                            wandb.log({'valid_rmse': valid_rmse
                                        , 'test_rmse': test_rmse
                                        , 'train_rmse': train_rmse
                                        # , 'orig_valid_rmse': orig_valid_rmse
                                        # , 'orig_test_rmse': orig_test_rmse
                                        # , 'orig_train_rmse': orig_train_rmse 
                                        })     
                        if valid_rmse < best_valid_rmse:
                            best_valid_rmse = valid_rmse
                            best_test_rmse = test_rmse
                            best_train_rmse = train_rmse
                            
                            #get the run id from wandb
                            run_id = wandb.run.id
                            torch.save(model.state_dict(), f'models/mjolnir4/SAINT_model_best_{run_id}.pt')
                            #Save as artifact on weights and biases
                            artifact = wandb.Artifact(f'SAINT_model_best_{run_id}', type='model')
                            #add the run id to the artifact                       
                            wandb.run.log_artifact(artifact)
                            early_stop_count = 0
                        else:
                            early_stop_count += 1
                            print(f"Early stopping counter: {early_stop_count}/{opt['patience']}")
                            if early_stop_count >= opt['patience']:
                                print('EARLY STOPPING')
                                break
                    model.train()
        
        del trainloader
        del validloader
        del testloader
        del model


In [1]:
import pandas as pd
df = pd.read_parquet('df_post.parquet')


In [2]:
df['Acc_GrossClaim'].isna().sum()

0

In [ ]:
1276544/6050571

In [ ]:
df.shape

In [ ]:
df['Acc_GrossClaim'].fillna(0, inplace=True)

In [ ]:
df.to_parquet('df_post.parquet')